In [2]:
from buildstock_query import BuildStockQuery, ExeId, MappedColumn
import pandas as pd
import sqlalchemy as sa

In [3]:
my_run = BuildStockQuery(db_name='resstock_core',
                        table_name='sdr_magic17',
                        workgroup='rescore',
                        buildstock_type='resstock',
                        skip_reports=False)

INFO:buildstock_query.query_core:Loading sdr_magic17 ...
INFO:botocore.tokens:Loading cached SSO token for nrel-sso
INFO:buildstock_query.query_core:33 queries cache read from None.
INFO:buildstock_query.query_core:33 queries cache is updated.
INFO:buildstock_query.main:Getting Success counts...
INFO:buildstock_query.report_query:Checking integrity with ts_tables ...
INFO:buildstock_query.query_core:No new queries to save.


         success  inapplicable  fail   Sum  Applied %  bad-chng  ok-chng  \
upgrade                                                                    
0           17.0           0.0   0.0  17.0        0.0         0        0   
1           17.0           0.0   0.0  17.0      100.0         0       17   
10          15.0           2.0   0.0  17.0       88.2         0       15   
11          17.0           0.0   0.0  17.0      100.0         0       17   
12          17.0           0.0   0.0  17.0      100.0         0       17   
13          17.0           0.0   0.0  17.0      100.0         0       17   
14          15.0           0.0   2.0  17.0      100.0         0       15   
15          15.0           2.0   0.0  17.0       88.2         0       15   
16          14.0           3.0   0.0  17.0       82.4         0       14   
17          17.0           0.0   0.0  17.0      100.0         0       17   
18          17.0           0.0   0.0  17.0      100.0        17        0   
19          

In [3]:
query = my_run.get_building_ids(get_query_only=True)
query

'SELECT sdr_magic17_baseline.building_id \nFROM sdr_magic17_baseline'

In [4]:
building_type_map = {"Mobile Home": "MH", "Single-Family Detached": "SF",
                     "Single-Family Attached": "SF", "Multi-Family with 2 - 4 Units": "MF",
                     "Multi-Family with 5+ Units": "MF",}
bldg_col = my_run._get_column('build_existing_model.geometry_building_type_recs')
simple_bldg_col = MappedColumn(bsq=my_run, name='simple_bldg_type', mapping_dict=building_type_map,
                               key=bldg_col)
mapped_agg_query = my_run.query(enduses=['fuel_use_electricity_total_m_btu'],
                                       group_by=[simple_bldg_col],
                                          get_query_only=False)
print(mapped_agg_query)

INFO:botocore.tokens:Loading cached SSO token for nrel-sso


  simple_bldg_type  sample_count   units_count  \
0               MF             1  8.214531e+06   
1               MH             4  3.285812e+07   
2               SF            12  9.857437e+07   

   fuel_use_electricity_total_m_btu  
0                      1.108633e+08  
1                      5.676734e+08  
2                      4.080141e+09  


In [9]:
my_run.execute(f"""select "build_existing_model.usage_level" from {my_run.bs_table.name}""")

,build_existing_model.usage_level
0,High
1,Medium
2,Medium
3,Low
4,Low
5,High
6,Medium
7,Medium
8,Medium
9,High


In [13]:
usage_weight = {"High": 1.2, 
                    "Medium": 1,
                    "Low": 0.8 ,
                    "None": 0}
bldg_col = my_run._get_column('build_existing_model.usage_level')
impact_col = MappedColumn(bsq=my_run, name='usage_weight', mapping_dict=usage_weight,
                               key=bldg_col)
mapped_agg_query = my_run.query(enduses=[impact_col],get_query_only=False)
print(mapped_agg_query)

   sample_count  units_count  usage_weight
0            17  139647020.0  1.380041e+08


In [14]:
baseline_agg_query = my_run.query(enduses=['fuel_use_electricity_total_m_btu'],
                                       group_by=['geometry_building_type_recs'],
                                          get_query_only=True,
                                          )
print(baseline_agg_query)



SELECT sdr_magic17_baseline."build_existing_model.geometry_building_type_recs" AS geometry_building_type_recs, sum(1) AS sample_count, sum(sdr_magic17_baseline."build_existing_model.sample_weight") AS units_count, sum(sdr_magic17_baseline."report_simulation_output.fuel_use_electricity_total_m_btu" * sdr_magic17_baseline."build_existing_model.sample_weight") AS fuel_use_electricity_total_m_btu 
FROM sdr_magic17_baseline 
WHERE sdr_magic17_baseline.completed_status = 'Success' GROUP BY 1 ORDER BY 1


In [15]:
baseline_agg = my_run.execute(baseline_agg_query)
baseline_agg

,geometry_building_type_recs,sample_count,units_count,fuel_use_electricity_total_m_btu
0,Mobile Home,4,3.285812e+07,5.676734e+08
1,Multi-Family with 2 - 4 Units,1,8.214531e+06,1.108633e+08
2,Single-Family Detached,12,9.857437e+07,4.080141e+09


In [14]:
baseline_agg_query2 = my_run.query(enduses=['fuel_use_natural_gas_total_m_btu'],
                                       group_by=['geometry_building_type_recs'],
                                          get_query_only=True)
print(baseline_agg_query2)

SELECT sdr_magic17_baseline."build_existing_model.geometry_building_type_recs" AS geometry_building_type_recs, sum(1) AS sample_count, sum(sdr_magic17_baseline."build_existing_model.sample_weight") AS units_count, sum(sdr_magic17_baseline."report_simulation_output.fuel_use_natural_gas_total_m_btu" * sdr_magic17_baseline."build_existing_model.sample_weight") AS fuel_use_natural_gas_total_m_btu 
FROM sdr_magic17_baseline 
WHERE sdr_magic17_baseline.completed_status = 'Success' GROUP BY 1 ORDER BY 1


In [ ]:
bq = my_run.submit_batch_query([baseline_agg_query, baseline_agg_query2])
print(bq)
my_run.get_batch_query_report(bq)

1


INFO:buildstock_query.query_core:Submitted queries[0] (CACHED)


{'submitted': 0, 'running': 0, 'pending': 2, 'completed': 0, 'failed': 0}

INFO:botocore.tokens:Loading cached SSO token for nrel-sso
INFO:buildstock_query.query_core:Submitted queries[1] (778f3a7e-3f5c-4114-8e74-1eebd2015ee4)


In [16]:
my_run.get_batch_query_report(bq)

INFO:botocore.tokens:Loading cached SSO token for nrel-sso


{'submitted': 2, 'running': 0, 'pending': 0, 'completed': 2, 'failed': 0}

In [17]:
result_df = my_run.get_batch_query_result(bq, combine=True)
result_df

INFO:buildstock_query.query_core:{'submitted': 2, 'running': 0, 'pending': 0, 'completed': 2, 'failed': 0}
INFO:buildstock_query.query_core:Batch query completed. 
INFO:buildstock_query.query_core:Got result from Query [0] (CACHED)
INFO:buildstock_query.query_core:Got result from Query [1] (778f3a7e-3f5c-4114-8e74-1eebd2015ee4)
INFO:buildstock_query.query_core:778f3a7e-3f5c-4114-8e74-1eebd2015ee4 cost 0.0 GB ($0.0). Session total: 0.0 GB ($0.0)
INFO:buildstock_query.query_core:Concatenating the results.


,geometry_building_type_recs,sample_count,units_count,fuel_use_electricity_total_m_btu,query_id,fuel_use_natural_gas_total_m_btu
0,Mobile Home,4,3.285812e+07,5.676734e+08,0,NaN
1,Multi-Family with 2 - 4 Units,1,8.214531e+06,1.108633e+08,0,NaN
2,Single-Family Detached,12,9.857437e+07,4.080141e+09,0,NaN
0,Mobile Home,4,3.285812e+07,NaN,1,2.084026e+08
1,Multi-Family with 2 - 4 Units,1,8.214531e+06,NaN,1,0.000000e+00
2,Single-Family Detached,12,9.857437e+07,NaN,1,6.231329e+09


In [18]:
query1 = my_run.query(enduses=['fuel_use_natural_gas_total_m_btu'], group_by=['state'],
                                              get_query_only=True)
query2 = my_run.query(enduses=['fuel_use_electricity_total_m_btu'],
                                              group_by=['state'],
                                              get_query_only=True)
exe_id1, future1 = my_run.execute(query1, run_async=True)
exe_id2, future2 = my_run.execute(query2, run_async=True)

assert exe_id1 != "CACHED"  # Commenting this line will cause the type checker to complain on the next line
my_run.get_query_output_location(exe_id1)

's3://resstock-core/athena_query_results/592a38b7-daa8-435f-b2eb-34bbb2354947.csv'

In [19]:
future1.as_pandas()

,state,sample_count,units_count,fuel_use_natural_gas_total_m_btu
0,CA,13104960,1.076511e+14,4.964261e+15
1,FL,26209920,2.153022e+14,9.928521e+15
2,IL,9530880,7.829171e+13,3.610371e+15
3,MD,13104960,1.076511e+14,4.964261e+15
4,MN,38123520,3.131668e+14,1.444149e+16
5,NC,26209920,2.153022e+14,9.928521e+15
6,NJ,39314880,3.229533e+14,1.489278e+16
7,OR,13104960,1.076511e+14,4.964261e+15
8,SD,12509280,1.027579e+14,4.738612e+15
9,TX,13104960,1.076511e+14,4.964261e+15


In [20]:
my_run.get_query_status(ExeId(exe_id1))

'SUCCEEDED'

In [21]:
my_run.get_all_running_queries()

[]

In [19]:
my_run.stop_all_queries()

INFO:buildstock_query.query_core:Stopped 0 queries


In [20]:
my_run.stop_query(ExeId(exe_id1))

{'ResponseMetadata': {'RequestId': '2b34fdc1-3a41-4904-b09b-6185bee959cb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 08 May 2024 22:03:02 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2b34fdc1-3a41-4904-b09b-6185bee959cb'},
  'RetryAttempts': 0}}

In [23]:
my_run.delete_everything()

Will delete the following tables ['res_test_03_2018_10k_20220607_baseline', 'res_test_03_2018_10k_20220607_timeseries', 'res_test_03_2018_10k_20220607_upgrades'] and the s3:/euss/tests/res_test_03_2018_10k_20220607 folder
Abandoned the idea.


In [20]:
# Mix and match: baseline from a different run from upgrades
my_run2 = BuildStockQuery(db_name='largeee_test_runs',
                          workgroup='largeee',
                          table_name=("baseline_20230525_baseline","cateory_2_20230525_timeseries","cateory_2_20230525_upgrades"),
                          buildstock_type='resstock',
                          skip_reports=False)


INFO:buildstock_query.query_core:Loading ('baseline_20230525_baseline', 'cateory_2_20230525_timeseries', 'cateory_2_20230525_upgrades') ...
INFO:botocore.tokens:Loading cached SSO token for nrel-sso
INFO:buildstock_query.main:Getting Success counts...
INFO:botocore.tokens:Loading cached SSO token for nrel-sso
INFO:buildstock_query.report_query:Checking integrity with ts_tables ...


         fail  unapplicable  success   Sum  Applied %  no-chng  bad-chng  \
upgrade                                                                    
0           7             0     9931  9938        0.0        0         0   
1           0             0     9931  9931      100.0        4       149   
2           0             0     9931  9931      100.0        3       391   
3           0          1793     8138  9931       81.9        4        16   
4           0           322     9609  9931       96.8      123        17   
5           0             0     9931  9931      100.0        2       222   

         ok-chng  true-bad-chng  true-ok-chng  null   any  no-chng %  \
upgrade                                                                
0              0              0             0     0     0        0.0   
1           9778            112          9815     0  9931        0.0   
2           9537            292          9637     0  9931        0.0   
3           8118              6

INFO:buildstock_query.query_core:12 queries cache saved to .bsq_cache/8d909ebc61709306982e1f11763d1184dfe3750c16c62b6aa305c9171b71760e_query_cache.pkl


All buildings are verified to have the same number of (8760) timeseries rows.
